In [17]:
import pandas as pd
import pyodbc
import os
import datetime as dt
import shutil

In [18]:
# retrieve shortage qty from SQL, get unique list of FV/ODM in the past 6 months
# compare with the FV/ODM list on the very day, get the difference list
# amend qty 0 for the different list

In [19]:
home = os.path.expanduser('~')
today = dt.datetime.today().date()
# retrieve date of 180 days ago
target_date = today - dt.timedelta(days = 180)

In [20]:
# # connect SQL server
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
# query statement
# select shortage data for the past 6 months with shortage qty not equal to zero
Query = 'SELECT [ODM],[Item],[Commodity],[FV],[Platform],[HP_PN],[BuyerName] FROM OPS.GPS_tbl_ops_shortage_ext WHERE ReportDate >= ? AND [Total Shortage Qty] != 0'
params = (target_date)
cursor.execute(Query, params)
shortage = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# shortage.columns = ['ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN','BuyerName']
cursor.close()
conn.close()

In [21]:
print(len(shortage))

21778


In [22]:
shortage['len'] = shortage['HP_PN'].apply(lambda x: len(x))
shortage.sort_values(by = 'len', inplace = True, ascending=True)
shortage.drop(columns = 'len', inplace = True)
shortage = shortage.drop_duplicates(keep = 'last')
print(len(shortage))

8191


In [23]:
shortage = shortage.drop_duplicates(subset = ['ODM','Commodity','FV'],keep = 'last')
print(len(shortage))
shortage_tmp = shortage[['ODM','Commodity','FV']]

1271


In [24]:
today_shortage = pd.read_excel(os.path.join(home, 'Desktop', 'shortage_all.xlsx'), sheet_name='Sheet1' )
today_shortage = today_shortage[['ODM','Commodity','FV']]

In [25]:
merge = shortage_tmp.merge(today_shortage, on = ['ODM','Commodity','FV'], how = 'outer', indicator= True)

In [26]:
print(len(merge[merge['_merge'] == 'left_only']))
print(len(merge[merge['_merge'] == 'right_only']))
print(len(merge[merge['_merge'] == 'both']))

966
40
372


In [27]:
amend_data = merge[merge['_merge'] == 'left_only'].drop(columns = '_merge').reset_index(drop = True)
amend_data = amend_data.merge(shortage, on = ['ODM','Commodity','FV'], how = 'left')
# amend_data['Total Shortage Qty'] = 0
amend_data['ReportDate'] = today
amend_data['last FD date'] = today
amend_list = ['P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside']
for _ in amend_list:
    amend_data[_] = 0


In [28]:
amend_data.head()

,ODM,Commodity,FV,Item,Platform,HP_PN,BuyerName,ReportDate,last FD date,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside
0,CQQCI,CPU,INTEL ALDERLAKE-U 7305,44.0,Digdug/RONIN14,N03113-N07,Rocky,2024-07-02,2024-07-02,0,0,0,0,0,0
1,CQIEC,POWERCORDS,PWRCORD C13 1.83M STKR CON INDIA GREY,10.0,BOGGLE,L97621-124,LinSharon,2024-07-02,2024-07-02,0,0,0,0,0,0
2,WHFXN,CardReader,4TB 7200 SATA ENT 3.5,19,Flatiron,M93943-001,TangWinnie,2024-07-02,2024-07-02,0,0,0,0,0,0
3,CQQCI,LCD,14WQXGA AG UWVA EDPPSR BENTNB DCI 120Hz 500N,20.0,Olaf 14 93H34AV/Oaken 14 93H35AV,M75057-392,FerngApple,2024-07-02,2024-07-02,0,0,0,0,0,0
4,CEI,Memory,LPDDR5 32Gb 6400 0.5v IC FBGA315,DDR411,Park 24C1/Thompson 24C1/Motto 24C1,M52547-392,JhangEmily,2024-07-02,2024-07-02,0,0,0,0,0,0


In [29]:
amend_data.to_excel(os.path.join(home, 'amend_stge_zero.xlsx'))

In [32]:
# upload amend data to SQL
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

for index, row in amend_data.iterrows():
    s_ODM = row['ODM']
    s_Item = row['Item']
    s_Commodity = row['Commodity']
    s_FV = row['FV']
    s_Platform = row['Platform']
    s_P1 = row['P1']
    s_P2 = row['Net P2']
    s_P3 = row['Net P3']
    s_Total = row['Total Shortage Qty']
    s_BT = row['BT shortage']
    s_working = row['Working on upside']
    s_ReportDate = pd.to_datetime(row['ReportDate'])
    s_lastFDdate = pd.to_datetime(row['last FD date'])
    s_BuyerName = row['BuyerName']
    s_PN = row['HP_PN']
    try:
        cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_shortage_zero ( ODM,Item,Commodity,FV,Platform,P1,[Net P2],[Net P3],[Total Shortage Qty],[BT shortage],[Working on upside],ReportDate,[last FD date],HP_PN,BuyerName )\
                        VALUES('{s_ODM}','{s_Item}','{s_Commodity}','{s_FV}','{s_Platform}','{s_P1}','{s_P2}','{s_P3}','{s_Total}','{s_BT}','{s_working}','{s_ReportDate}','{s_lastFDdate}','{s_PN}','{s_BuyerName}')".replace("'NaT'", "NULL"))
    except:
        print(f'Error on {s_ODM},{s_Item},{s_Commodity},{s_FV},{s_Platform},{s_P1},{s_P2},{s_P3},{s_Total},{s_BT},{s_working},{s_ReportDate},{s_lastFDdate},{s_PN},{s_BuyerName}')

conn.commit()
conn.close()

Error on CQIEC,1,ODD,ODD DVD SATA 9.5mm Slim Tray,BOGGLE/BoggleR,0,0,0,0,0,0,2024-07-02 00:00:00,2024-07-02 00:00:00,849054-6C7,ManeeratKosit(Kosit-GPSThailand)


In [ ]:
# 超過180以前等於0的要刪掉，不然會無限長大
# delte shortage = 0 where report date is before 180 days, otherwise, the database will grow up without limit

In [ ]:
# # # connect SQL server
# conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
# cursor = conn.cursor()
# # query statement
# Query = 'DELETE FROM OPS.GPS_tbl_ops_shortage_ext WHERE ReportDate <= ? AND [Total Shortage Qty] = 0'
# params = (target_date)
# cursor.execute(Query, params)
# shortage = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# # shortage.columns = ['ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN','BuyerName']
# cursor.close()
# conn.close()